# Asset数据集使用示例

In [1]:
import os, sys, argparse, logging

import pandas as pd
import numpy as np
from datetime import date

# 首先将framework的目录加进sys.path中，方便使用相对路径导入
framework_path = os.path.join(os.path.abspath(''), '../framework')
if framework_path not in sys.path:
    sys.path.append(framework_path)

In [2]:
# 导入component包
# 所有服务均可以使用getComponentModule动态导入

from import_func import *

asset_module = getComponentModule('Asset')
group_module = getComponentModule('Group')

init YamlSvc
init YamlSvc
init ConstantSvc
init IndicatorSvc
init RawDataSvc
init SqlRawDataSvc
init DbConnector
init TableColumnSvc
init LocalRawDataSvc
init DateSvc
raw_data_svc.sqlQuery: SELECT TRADE_DAYS FROM asharecalendar WHERE S_INFO_EXCHMARKET='SSE' ORDER BY TRADE_DAYS


## 一、资产组的使用

In [3]:
# 获取Group类，component包的导入与svc不同，主要原因为1、svc是单例，可以直接获取实体；2、compon中的类可能需要经常扩展，每新增一个类后修改配置文件比较麻烦
Group = group_module.Group

# 新建一个组作为根节点
root = Group('root')

# 打印root的类型
print('*' * 50)
print('type(root): \n', type(root))

# print方法打印组的层级结构
print('*' * 50)
print('root.print():')
root.print()

**************************************************
type(root): 
 <class 'component.asset.group.Group'>
**************************************************
root.print():
group: root


In [4]:
# 新建三层的层级结构，使用addChildGroup增加子组
root = Group('root')
g1 = Group('g1')
g2 = Group('g2')
root.addChildGroup(g1)
g1.addChildGroup(g2)

print('*' * 50)
root.print()

# 任意一个组都可以打印其本身和children的结构
print('*' * 50)
g1.print()

**************************************************
group: root
	group: g1
		group: g2
**************************************************
group: g1
	group: g2


In [5]:
# 获取所有子组
print('*' * 50)
print([group.getName() for group in root.getChildGroup()])

# 循环获取所有子组
print('*' * 50)
print([group.getName() for group in root.getAllGroup()])

**************************************************
['g1']
**************************************************
['root', 'g1', 'g2']


In [6]:
# 通过路径或列表访问组
print('*' * 50)
print(root.getGroup('g1/g2').getName())

print('*' * 50)
print(root.getGroup(['g1', 'g2']).getName())

print('*' * 50)
print(g1.getGroup('g2').getName())

# 路径为空时返回自己
print('*' * 50)
print(g1.getGroup([]).getName())

**************************************************
g2
**************************************************
g2
**************************************************
g2
**************************************************
g1


## 二、为资产组添加资产

In [12]:
# 获取资产类
Asset = asset_module.Asset
IndexAsset = asset_module.IndexAsset

# Asset类为普通资产，需要自己设置原始数据
random_asset = Asset('random')
random_nav = pd.Series(np.random.rand(500)/10+1, index=pd.date_range(date(2020,1,1), periods=500))
random_asset.setRawNavdata(random_nav)

print('*' * 50)
print(random_asset.getRawNavData())

# IndexAsset为指数资产，可以自动获取原始数据， 此外还有股票资产和指数资产等
# 以上证指数为例
index_000001 = IndexAsset('000001', '000001.SH')
print('*' * 50)
print(index_000001.getRawNavData())


**************************************************
2020-01-01    1.077553
2020-01-02    1.005979
2020-01-03    1.064148
2020-01-04    1.033949
2020-01-05    1.097988
                ...   
2021-05-10    1.079174
2021-05-11    1.008559
2021-05-12    1.083126
2021-05-13    1.051372
2021-05-14    1.009072
Freq: D, Length: 500, dtype: float64
**************************************************
TRADE_DT
1990-12-19     100.0000
1990-12-20     104.3900
1990-12-21     109.1300
1990-12-24     114.5500
1990-12-25     120.2500
                ...    
2022-07-19    3279.4313
2022-07-20    3304.7243
2022-07-21    3272.0012
2022-07-22    3269.9739
2022-07-25    3250.3880
Name: S_DQ_CLOSE, Length: 7720, dtype: float64


In [11]:
# 裁切数据
# 设置id_date时，同时指定裁切数据的参数，参考datesvc
index_000001.setIdDate(date(2021, 5, 1), 100)
print('*' * 50)
print(index_000001.getUsableNavData())

# 获取指标，封装indicator_svc中的指标
print('*' * 50)
print(index_000001.getSharpe(index_000001.getUsableNavData()))


**************************************************
TRADE_DAYS
2020-12-03    3442.1359
2020-12-04    3444.5814
2020-12-07    3416.6037
2020-12-08    3410.1771
2020-12-09    3371.9640
                ...    
2021-04-26    3441.1658
2021-04-27    3442.6111
2021-04-28    3457.0683
2021-04-29    3474.9011
2021-04-30    3446.8564
Length: 100, dtype: float64
**************************************************
0.00681503793397916


In [9]:
# 为之前设置的资产组添加资产
g1.addChildAsset(random_asset)
g2.addChildAsset(index_000001)
print('*' * 50)
root.print()

# 访问所有叶子资产
print('*' * 50)
all_asset = root.getAllAsset()
print([asset.getName() for asset in all_asset])

# 使用路径访问某个资产
print('*' * 50)
print(root.getAsset('g1/g2/000001').getName())

**************************************************
group: root
	group: g1
		asset: random
		group: g2
			asset: 000001
**************************************************
['random', '000001']
**************************************************
000001


## 三、组和资产的其他操作

In [16]:
# 设置组和和资产的权重范围

# 默认权重范围为0-1
print('*' * 50)
print('root.getWeightRange(): ')
print(root.getWeightRange())
print('*' * 50)
print('random_asset.getWeightRange(): ')
print(random_asset.getWeightRange())

# 设置权重
root.setWeightRange([0.2, 0.4])
random_asset.setWeightRange([0.1, 0.8])

print('*' * 50)
print('root.getWeightRange(): ')
print(root.getWeightRange())
print('*' * 50)
print('random_asset.getWeightRange(): ')
print(random_asset.getWeightRange())

**************************************************
root.getWeightRange(): 
[0.2, 0.4]
**************************************************
random_asset.getWeightRange(): 
[0.1, 0.8]
**************************************************
root.getWeightRange(): 
[0.2, 0.4]
**************************************************
random_asset.getWeightRange(): 
[0.1, 0.8]


In [22]:
# 复制组和资产

root_copy = root.copy()
print('*' * 50)
root_copy.print()

random_asset_copy = random_asset.copy()

# 拷贝为深拷贝
print('*' * 50)
print('id(root): {}, id(root_copy): {}'.format(id(root), id(root_copy)))

print('*' * 50)
print('id(random_asset): {}, id(random_asset_copy): {}'.format(id(random_asset), id(random_asset_copy)))

print('*' * 50)
print('id(root.getAsset(\'g1/g2/000001\')): {}'.format(id(root.getAsset('g1/g2/000001'))))
print('id(root_copy.getAsset(\'g1/g2/000001\')): {}'.format(id(root_copy.getAsset('g1/g2/000001'))))

**************************************************
group: root
	group: g1
		asset: random
		group: g2
			asset: 000001
**************************************************
id(root): 2289496435440, id(root_copy): 2290596736928
**************************************************
id(random_asset): 2289502421488, id(random_asset_copy): 2290596735248
**************************************************
id(root.getAsset('g1/g2/000001')): 2291082701744
id(root_copy.getAsset('g1/g2/000001')): 2290596733664
